In [2]:
import fxcmpy, socketio, engineio
print(fxcmpy.__version__)
TOKEN = "5b6de8dc3b260ab0fab6e007bf21c43ee4fc7a27"
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level="error")
con

1.3.2


In [47]:
import datetime as dt
from icecream import ic 
import socketio, engineio
from django.conf import settings

start = dt.datetime(2022,5,1)
end = dt.datetime(2022,5,15)
data = con.get_candles('EUR/USD', period='m1', start=start, end=end) #number=250 start=start, end=end
# data.to_csv("mar.csv")
connection = con
data

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2022-06-01 05:52:00,1.07079,1.07085,1.07085,1.07073,1.07092,1.07098,1.07100,1.07088,122
2022-06-01 05:53:00,1.07086,1.07084,1.07090,1.07083,1.07099,1.07097,1.07103,1.07097,79
2022-06-01 05:54:00,1.07085,1.07091,1.07094,1.07084,1.07097,1.07105,1.07108,1.07097,84
2022-06-01 05:55:00,1.07091,1.07084,1.07095,1.07084,1.07104,1.07097,1.07109,1.07097,72
2022-06-01 05:56:00,1.07084,1.07096,1.07097,1.07076,1.07098,1.07108,1.07109,1.07089,67
...,...,...,...,...,...,...,...,...,...
2022-06-01 09:57:00,1.07232,1.07242,1.07243,1.07224,1.07244,1.07254,1.07255,1.07239,129
2022-06-01 09:58:00,1.07241,1.07254,1.07258,1.07237,1.07254,1.07267,1.07271,1.07250,144
2022-06-01 09:59:00,1.07253,1.07261,1.07262,1.07246,1.07267,1.07275,1.07275,1.07261,148


In [75]:
from datetime import datetime
import time, os, json

In [76]:
class OrderBook:

    def __init__(self):
        self.book_name = f"order_book_{str(datetime.today().date())}.json"

    def _update_order_book(self, data:list=None):
        default_data = {
            "book_date": str(datetime.today().date()),
            "orders": data
        }
        with open(self.book_name, "w+") as order_book:
            json.dump(default_data, order_book, default=str)
        return True

    def add_new_order(self, entry_order, order_type="ENTRY", exit_signal=None):
        if not os.path.exists(self.book_name):
           self._update_order_book()
        else:
            book_data = self.read_order_book()
            
            if not book_data:
                book_data = {
                    entry_order.time : [
                        entry_order,
                        order_type,
                        exit_signal
                    ]
                }
                self._update_order_book(book_data)
                return True
            else:
                if order_type.lower() == "entry":
                    if is_order_in_book(entry_order):
                        return False
                    book_data[entry_order.time] = [
                        entry_order,
                        order_type,
                        exit_signal
                    ]
                    self._update_order_book(book_data)
                    return True
                elif order_type.lower() == "exit":
                    if is_order_in_book(entry_order):
                        order = order_book[entry_order.time]
                        if order[2] == exit_signal:
                            return False
                        book_data[entry_order.time] = [
                            entry_order,
                            order_type,
                            exit_signal
                        ]
                        self._update_order_book(book_data)
                        return True
                    return False
            return False
                    

    def read_order_book(self):
        if not os.path.exists(self.book_name):
            self._update_order_book()
        with open(self.book_name, "r") as order_book:
            book = json.load(order_book)
        return book["orders"]

    def is_order_in_book(self, entry_order):
        book_data = self.read_order_book()
        if book_data:
            if entry_order.time in book_data.keys():
                True
        return False

    def get_last_order(self):
        if not os.path.exists(self.book_name):
            return None
        book_data = self.read_order_book()
        if not book_data:
            return None
        else:
            pass
            # Put code for sorting
        
    def is_last_order_open(self):
        last_order = self.get_last_order()
        if last_order and last_order[2] is None:
            return True
        return False
    

In [77]:
def start_entry_trading(entry,  **kwargs):
    # time.sleep(10)
    df = connection.get_candles('EUR/USD', period='m1', number=250)
    entry_signal = FindEntry().find_entry(entry, df, **kwargs)
    order_book = OrderBook()
    if entry_signal:
        # entry_signal 
        last_order = order_book.get_last_order()
        if last_order:
            if last_order_df.entry_signal == entry_signal.entry_signal and order_book.add_new_order(entry_signal):
                print("Entry signal found placing a new order")
                # order = connection.create_entry_order(
                #             symbol='EUR/USD',
                #             is_buy=True if entry_signal.entry_signal == "BUY" else False,
                #             rate=entry_signal.entry_price,
                #             is_in_pips=False,
                #             time_in_force='GTD',
                #         )
        else:
            ic(order_book.is_last_order_open())
            if order_book.is_last_order_open():
                pass
            else:
                if order_book.add_new_order(entry_signal):
                    print("Entry signal found placing a new order")
                    # order = connection.create_entry_order(
                    #     symbol='EUR/USD',
                    #     is_buy=True if entry_signal.entry_signal == "BUY" else False,
                    #     rate=entry_signal.entry_price,
                    #     is_in_pips=False,
                    #     time_in_force='GTD',
                    # )
        print("Some thing else happend")

start_entry_trading(SMAStrategy3Level1, stoploss=0.0005, target=0.0008)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
import pandas as pd
def print_data(data, dataframe):
    print('%3d | %s | %s, %s, %s, %s, %s'
          % (len(dataframe), data['Symbol'],
             pd.to_datetime(int(data['Updated']), unit='ms'),
             data['Rates'][0], data['Rates'][1], data['Rates'][2],
             data['Rates'][3]))

def arraySortedOrNot(arr):
 
    # Calculating length
    n = len(arr)
 
    # Array has one or no element or the
    # rest are already checked and approved.
    if n == 1 or n == 0:
        return True
 
    # Recursion applied till last element
    return arr[0] <= arr[1] and arraySortedOrNot(arr[1:])
# data.to_csv("feb.csv")

In [2]:
from icecream import ic
from datetime import datetime
import pandas_ta as pta
d1 = pd.read_csv("jan1.csv")
# d2 = pd.read_csv("dec2.csv")
# d3 = pd.read_csv("dec3.csv")
# d3 = pd.read_csv("second_jan.csv")
frames = [d1]
data = pd.concat(frames)
data.index = data.date.values

# # data = data.loc[(data["date"] > datetime.strptime("2022-01-05", "%Y-%m-%d"))]
data = data.drop(["date"], axis=1)
# data = data.iloc[40:3000] # 5831
data
# pd.DataFrame(index=data.date, data=data)


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
2022-01-02 22:05:00,1.13684,1.13684,1.13684,1.13680,1.13776,1.13776,1.13776,1.13776,7
2022-01-02 22:06:00,1.13680,1.13655,1.13680,1.13652,1.13776,1.13691,1.13777,1.13691,9
2022-01-02 22:07:00,1.13655,1.13675,1.13675,1.13655,1.13692,1.13699,1.13701,1.13691,14
2022-01-02 22:08:00,1.13675,1.13675,1.13675,1.13675,1.13698,1.13699,1.13699,1.13696,3
2022-01-02 22:09:00,1.13675,1.13675,1.13675,1.13675,1.13701,1.13701,1.13701,1.13701,1
...,...,...,...,...,...,...,...,...,...
2022-01-06 23:56:00,1.12946,1.12945,1.12947,1.12945,1.12959,1.12958,1.12959,1.12958,4
2022-01-06 23:57:00,1.12946,1.12953,1.12955,1.12946,1.12958,1.12966,1.12968,1.12958,38
2022-01-06 23:58:00,1.12953,1.12951,1.12953,1.12950,1.12967,1.12964,1.12967,1.12963,14
2022-01-06 23:59:00,1.12951,1.12942,1.12952,1.12942,1.12963,1.12955,1.12965,1.12955,41


In [5]:
# import ta
# from ta.trend import *
# from ta.momentum import *
from IPython.display import display


# data["SMALL_SMA"]=sma_indicator(data.bidclose, window=7)
# data["LARGE_SMA"]=sma_indicator(data.bidclose, window=10)
# # Stochastic need to fix
# data["STOCH"] = stoch(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["STOCH_SIGNAL"] = stoch_signal(data.bidhigh, data.bidlow, data.bidclose, window=7, smooth_window=3)
# data["RSI"] = rsi(data.bidclose)
# data["SMA_SIGNAL"] = 

# len(data.index)
# data = kada

In [25]:
import importlib
import strategy
importlib.reload(strategy)
from strategy import BaseStrategy
import pandas as pd
import numpy as np
from ta.trend import *
from ta.momentum import *
import copy



class SMAStrategy(BaseStrategy):
    name = "smarsistochastic_strategy"
    description = """7,10 SMA, 7,3,3 Stochastic and RSI Strategy"""
    
    def signal_found(self, data:dict):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "SELL"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "BUY"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        
        df.loc[(df["sma_signal"] == "BUY")  & (df["stoch"] >= 60) & (df["stoch_signal"] >= 60), "stoch_signal_f"] = "CHUTIYA"
        df.loc[(df["sma_signal"] == "SELL")  & (df["stoch"] <= 40) & (df["stoch_signal"] <= 40), "stoch_signal_f"] = "CHUTIYA" 
        df["entry_signal"] = df.sma_signal
        df["entry_price"] = df.bidopen
        return self.signal_found(df.loc[(df["stoch_signal_f"]== "CHUTIYA")].tail(1))
        
    
    def apply_indicator(self):
        super(SMAStrategy, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["stoch"] = round(stoch(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["stoch_signal"] = round(stoch_signal(close=self.dataframe.bidclose, low=self.dataframe.bidlow, high=self.dataframe.bidhigh, window=7, smooth_window=3),6)
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 7)


    
# strategy = SMAStrategy()
# data[:220]
# # for i in range(51,100):
# strategy.dataframe = data[:222]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# df = df.loc[(df["SMA_SIGNAL"]=="BUY") | (df["SMA_SIGNAL"]=="SELL")]
# dd

# rs_client.set("ch", pickle.dumps(SMAStrategy))

In [26]:
class SMAStrategy1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        df.loc[(df["small_sma"] < df["large_sma"]), "sma_signal"] = "DOWN_CROSSOVER"
        df.loc[(df["small_sma"] > df["large_sma"]), "sma_signal"] = "UP_CROSSOVER"
        
        def sma_signal_maker(sma_signal):
            """function required more optimized logic"""
            signals = dict(enumerate(list(sma_signal)))
            final_signals = []
            for k,v in signals.items():
                if k == 0:
                    final_signals.append(v)
                    continue
                if v == signals[k-1]:
                    final_signals.append("")
                else:
                    final_signals.append(v)
                
            return final_signals
        
        
        
        df["sma_signal"] = sma_signal_maker(df.sma_signal)
        # df["entry_signal"] = np.select([(df["sma_signal"] == "UP_CROSSOVER"),(df["sma_signal"] == "DOWN_CROSSOVER")],["BUY","SELL"]) 
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        new_df = df[last_sma_crossover.name:df.iloc[-1].name]
        new_df = new_df.iloc[1: , :]
        signal = None
        data = None
        amt = 0.00003
        if last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            candle_counter = 0
            iter_counter = 0
            signal = "SELL"
            for row in new_df.itertuples():
                if row.max_bid+amt >= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
                # ic(row)
            # sma_touched_df = new_df.head(2).loc[(new_df["max_bid"] >= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "SELL"
            # data = new_df.head(2).tail(1)
        elif last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            candle_counter = 0
            iter_counter = 0
            for row in new_df.itertuples():
                if row.min_bid-amt <= row.small_sma:
                    candle_counter = 0
                else:
                    candle_counter += 1
                if candle_counter >= 2 or iter_counter == 5:
                    break
                iter_counter += 1
            if candle_counter != 2:
                return None
            else:
                data = new_df[row.Index : row.Index]
            
            # sma_touched_df = new_df.head(2).loc[(new_df["min_bid"] <= new_df["small_sma"])]
            # if not sma_touched_df.empty:
            #     return None
            # signal = "BUY"
            # data = new_df.head(2).tail(1)
        if signal and not data.empty:
            # ic(signal)
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
        #     # ic(d)
            return d
        # ic(last_sma_crossover)
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        

# strategy = SMAStrategy1()
# strategy.dataframe = data #.iloc[120:187]
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).tail(50)
# dd

In [27]:
class SMAStrategy2(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER" and \
                    last_sma_crossover.min_bid > last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and \
                    last_sma_crossover.max_bid < last_sma_crossover.small_sma:
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        if signal and not data.empty:
            d_date = data.index[0]
            d = data.to_dict()
            d["entry_price"] = d["bidclose"]
            d["entry_signal"] = {d_date: signal}
            return d
        return None
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).iloc[1650:1700]

In [28]:
class SMAStrategy3Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""  
    
    def signal_found(self, data):
        return data.to_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        signal = self.entry_signal
        target = self.__class__.get_target_price(signal.entry_price, self.target, signal.entry_signal)
        stoploss = self.__class__.get_stoploss_price(signal.entry_price, self.stoploss, signal.entry_signal)
        df_last_date = df.tail(1).index[0]
        temp_df = df[:signal.Index][-10:]
        rows_min = min(temp_df.min_bid)
        rows_max = max(temp_df.max_bid)
        slp_price= 0.0002
        tgp_price = 0.0002
        new_df = df[signal.Index:df_last_date].iloc[1: , :][:100]
        exit_price = 0
        exit_time = None
        status = ""
        rsi_high_hit = 0
        rsi_low_hit = 0
        
        def last_prices_fetcher(price:float, storage:list):
            if len(storage) == 4:
                storage.append(price)
                storage.pop(0)
            else:
                storage.append(price)
            return storage
        
        def get_trend(price_list:list):
            avr = round(sum(price_list) / len(price_list), 5)
            # ic(price_list[-1] - avr)
            if price_list[-1] > avr:
                return "UP"
            elif price_list[-1] < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if signal.entry_signal == "BUY":
            last_row = None
            new_sl = min(signal.bidopen - slp_price, rows_min)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 70 and row.rsi < 70:
                    rsi_high_hit += 1
                if last_row.rsi < 30 and row.rsi > 30:
                    rsi_low_hit += 1
                
                # ic(last_row.rsi > 70 and row.rsi < 70)
                # ic(row.max_bid >= target)
                # ic(row)
                # ic(target)
                if new_tgp_row == None:
                    if row.max_bid >= signal.bidclose + tgp_price:
                        new_tgp_row = row  
                elif row.max_bid >= target:
                    target_hit = True
                    if row.rsi < 67:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.min_bid <= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose <= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi > 71:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi < 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                
                elif last_row.rsi > 70 and row.rsi < 70:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        elif signal.entry_signal == "SELL":
            last_row = None
            new_sl = max(rows_max, signal.bidopen + slp_price)
            new_tgp_row = None
            last_prices = []
            down_trend_counter = 0
            target_hit = False
            last_trend = ""
            for row in new_df.itertuples():
                last_prices_fetcher(row.bidclose, last_prices)
                trend = get_trend(last_prices)
                if last_row == None:
                    last_row = row
                if last_row.rsi > 30 and row.rsi < 30:
                    rsi_high_hit += 1
                if last_row.rsi < 70 and row.rsi > 70:
                    rsi_low_hit += 1
                if new_tgp_row == None:
                    if row.min_bid <= signal.bidclose - tgp_price:
                        new_tgp_row = row 
                elif row.min_bid <= target:
                    target_hit = True
                    if row.rsi > 33:
                        exit_price = row.bidclose
                        exit_time = row.Index
                        status = "TG"
                        break
                elif row.max_bid >= new_sl:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SLPR"
                    break
                elif row.bidclose >= stoploss:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "SL"
                    break
                # elif last_row.rsi < 29:
                #     tgp_broken = True
                # elif tgp_broken:
                #     if row.rsi > 50:
                #         exit_price = row.bidclose
                #         exit_time = row.Index
                #         status = "TGP"
                elif last_row.rsi < 30 and row.rsi > 30:
                    exit_price = row.bidclose
                    exit_time = row.Index
                    status = "TGP"
                    break
                last_row = row
                last_trend = trend
        if all([exit_price, status, exit_time]):
            return exit_price, exit_time, status
        return 0,0,0

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value
            
    

    
class SMAExitLevel1(BaseStrategy):
    
    def get_next_entry_signal(self):
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        df = self.dataframe.copy()
        new_df = df[self.entry_signal.Index:]
        if new_df.empty:
            return None
        # this is temporary code for testing it need optimization
        for date_index in new_df.index:
            strategy.dataframe = df[:date_index]
            strategy.is_valid_dataframe()
            signal = strategy.get_signal()
            if signal:
                signal_df = pd.DataFrame(signal).iloc[0]
                if signal_df.name <= self.entry_signal.Index:
                    continue
                return signal
        return None
    
    def strategy(self):
        df = self.dataframe.copy()
        stoploss_price = self.__class__.get_stoploss_price(self.entry_signal.entry_price, self.stoploss, self.entry_signal.entry_signal)
        exit_strategy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        exit_strategy.dataframe = df
        exit_strategy.entry_signal = self.entry_signal
        exit_strategy.is_valid_dataframe()
        exit_signal = exit_strategy.get_signal()
        
        # return exit_signal
        next_entry_signal = self.get_next_entry_signal()
        # ic(next_entry_signal)
        if next_entry_signal:
            next_entry_signal = pd.DataFrame(next_entry_signal).iloc[0]
        else:
            next_entry_signal = pd.DataFrame()
        
        if exit_signal == (0,0,0):
            new_df = self.dataframe[self.entry_signal.Index:]
            counter = 0
            for frame in new_df.itertuples():
                if self.entry_signal.entry_signal == "BUY" and frame.bidclose <= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
                elif self.entry_signal.entry_signal == "SELL" and frame.bidclose >= stoploss_price:
                    return frame.bidclose, frame.Index, "SL"
        elif exit_signal == (0,0,0) and not next_entry_signal.empty:
            next_entry_time = next_entry_signal.name
            signal_entry_time = self.entry_signal.Index
            if next_entry_time > signal_entry_time:
                if self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "SELL":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose < -0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "SELL" and next_entry_signal.entry_signal == "BUY":
#                     for time being exit all entry
                    return next_entry_signal.bidclose, next_entry_time, "TGP"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "BUY":
                    if self.entry_signal.bidclose - next_entry_signal.bidclose > 0.0002:
                        return next_entry_signal.bidclose, next_entry_time, "SLPR"
                elif self.entry_signal.entry_signal == "BUY" and next_entry_signal.entry_signal == "SELL":
                # for time being exit and take new entry      
                    return next_entry_signal.bidclose, next_entry_time, "SLPR"
        return exit_signal


# SMAExitLevel1Copy = copy.deepcopy(SMAExitLevel1)
# d["entry_signal"] = {'2021-12-01 10:25:00': 'SELL'}
# test_signal = next(pd.DataFrame(d).itertuples())
# tragedy = SMAStrategy3Exit(target=0.0010, stoploss=0.0005)
# kata = data[20:193]
# print(kata.iloc[-1].name)
# tragedy.dataframe = kata
# tragedy.entry_signal = test_signal
# tragedy.is_valid_dataframe()
# dd= tragedy.get_signal()
# ic(dd)
# test_signal
# df_signal = next(pd.DataFrame(signal).itertuples())
#             strategy.entry_signal = next(pd.DataFrame(signal).itertuples())
#             # # # tragedy.is_valid_dataframe()
#             dd= strategy.get_signal()
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1).head(50) .iloc[1:60]
# dd
# data.iloc[0].name

In [18]:
ss = pd.DataFrame(sma_backtest.signal_data)
from strategy import DefaultExitStrategy
# bd = ss.itertuples()
# p = next(bd)
# ic(p)
# for i in bd:
#     ic(i)
#     break
# for test_signal in ss.itertuples():
#     if pd.Timestamp(test_signal.Index) == pd.Timestamp("2021-12-01 01:21:00"):
#         break
# test_signal

def run_exit_strategy(exit_strategy, signal_df, main_df):
    exit_details = []
    exit_strategy = exit_strategy(stoploss=0.0005, target=0.0008)
    exit_strategy.dataframe = main_df
    signal_data = signal_df.itertuples()
    next(signal_data)
    exit_details.append((0,0,0))
    for signal in signal_data:
        exit_strategy.entry_signal = signal
        if exit_strategy.is_valid_dataframe():
            # if signal == test_signal:
            #     ic(signal)
            #     ic(exit_strategy.get_signal())
            exit_details.append(exit_strategy.get_signal())

    signal_df["exit_price"] = [detail[0] for detail in exit_details]
    signal_df["exit_time"] = [detail[1] for detail in exit_details]
    signal_df["status"] = [detail[2] for detail in exit_details]
    return signal_df
dp = run_exit_strategy(SMAExitLevel1, ss, test_data)
dp

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,max_bid,...,small_sma,large_sma,rsi,rsi_signal,sma_signal,entry_price,entry_signal,exit_price,exit_time,status
2022-05-01 22:19:00,1.05428,1.05474,1.05474,1.05428,1.05440,1.05486,1.05486,1.05441,91,1.05474,...,1.054510,1.054647,54.550935,UP_REVERSE,,1.05474,BUY,0.00000,0,0
2022-05-01 22:28:00,1.05366,1.05399,1.05401,1.05366,1.05381,1.05415,1.05416,1.05382,115,1.05401,...,1.054226,1.054364,36.720081,UP_REVERSE,,1.05399,BUY,1.05452,2022-05-01 22:56:00,TGP
2022-05-01 22:30:00,1.05365,1.05381,1.05390,1.05364,1.05380,1.05397,1.05403,1.05379,91,1.05390,...,1.053977,1.054162,36.112771,UP_REVERSE,,1.05381,BUY,1.05452,2022-05-01 22:56:00,TGP
2022-05-01 22:56:00,1.05455,1.05452,1.05456,1.05448,1.05470,1.05467,1.05471,1.05462,127,1.05456,...,1.054471,1.054382,69.550617,DOWN_REVERSE,,1.05452,SELL,1.05478,2022-05-01 23:09:00,SLPR
2022-05-01 23:54:00,1.05380,1.05392,1.05392,1.05370,1.05394,1.05408,1.05408,1.05385,110,1.05392,...,1.053990,1.054073,35.110071,UP_REVERSE,,1.05392,BUY,1.05457,2022-05-02 00:06:00,TGP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 19:49:00,1.03965,1.04001,1.04004,1.03965,1.03979,1.04013,1.04018,1.03978,123,1.04004,...,1.039779,1.039782,62.473651,UP_REVERSE,,1.04001,BUY,1.04026,2022-05-13 19:53:00,TGP
2022-05-13 19:53:00,1.04040,1.04026,1.04040,1.04024,1.04055,1.04040,1.04055,1.04038,83,1.04040,...,1.040069,1.039980,65.543356,DOWN_REVERSE,,1.04026,SELL,1.04061,2022-05-13 20:45:00,SLPR
2022-05-13 20:40:00,1.04045,1.04037,1.04045,1.04037,1.04059,1.04052,1.04059,1.04051,16,1.04045,...,1.040327,1.040281,67.323014,DOWN_REVERSE,,1.04037,SELL,1.04088,2022-05-13 20:58:00,SL
2022-05-13 20:43:00,1.04050,1.04039,1.04050,1.04038,1.04066,1.04056,1.04066,1.04056,29,1.04050,...,1.040401,1.040365,60.322815,DOWN_REVERSE,,1.04039,SELL,0.00000,0,0


In [31]:
import strategy
importlib.reload(strategy)
class SMAStrategy3Level1(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    
    def signal_found(self, data):
        return data.to_dict()
    
    
    def create_signal(self, data, **kwargs):
        d_date = data.index[0]
        d = data.to_dict()
        d["entry_price"] = d["bidclose"]
        d["entry_signal"] = {d_date: kwargs.get("signal")}
        current_signal_df = pd.DataFrame(d).iloc[0]
        return d
    
    def strategy1(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        last_entry_signal = self.get_last_signal()
        last_entry_signal = next(pd.DataFrame(last_entry_signal).itertuples())
        tragedy = SMAStrategy3Exit(target=self.target, stoploss=self.stoploss)
        tragedy.dataframe = df
        tragedy.entry_signal = last_entry_signal
        tragedy.is_valid_dataframe()
        last_exit_signal= tragedy.get_signal()
        # ic(last_exit_signal)
        # ic(last_entry_signal.Index, last_entry_signal.bidclose)
        def get_trend(df):
            last_row_price = df.iloc[0].bidclose
            avr = round(df.iloc[:6].bidclose.sum() / 6, 5)
            if last_row_price > avr:
                return "UP"
            elif last_row_price < avr:
                return "DOWN"
            else:
                return "SIDE"
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        sma_crossover_counter = 0
        try:
            last_crossover = new_df.loc[(new_df["sma_signal_1"] == "UP_CROSSOVER") | (new_df["sma_signal_1"] == "DOWN_CROSSOVER")].iloc[-0]
        except:
            last_crossover = pd.DataFrame()
        for row in new_df.itertuples():
            if row.sma_signal_1.endswith("CROSSOVER"):
                break
            else:
                sma_crossover_counter += 1
                
        if not data.empty and signal == last_entry_signal.entry_signal:
            if "SLPR" in last_exit_signal:
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1) < 13:
                    return None
                elif not last_crossover.empty:
                    if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                        return None
                    elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                        return None
            elif last_exit_signal == (0,0,0):
                if (pd.Timestamp(data.index[0]) - pd.Timestamp(last_entry_signal.Index)) / pd.Timedelta(minutes=1) < 13:
                    return None

        if last_exit_signal[2] == "TG" and not data.empty:
            entry_diff = (pd.Timestamp(data.iloc[0].name) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if signal != last_entry_signal.entry_signal and entry_diff <= 15:
                ic(last_exit_signal)
                ic(last_entry_signal)
                return self.create_signal(data, signal=last_entry_signal.entry_signal)
        
        
        if not last_crossover.empty:
            if signal == "SELL" and last_crossover.sma_signal_1 == "UP_CROSSOVER" and sma_crossover_counter > 14:
                return None
            elif signal == "BUY" and last_crossover.sma_signal_1 == "DOWN_CROSSOVER" and sma_crossover_counter > 14:
                return None
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
       
    
    def get_previous_signal(self):
        negative_looper = list(range(-9,-1))[::-1]
        first_num = negative_looper[0]
        for n in negative_looper[1:]:
            signal = self.get_current_signal(last_signal_value=first_num, second_last_signal_value=n)
            if signal:
                return signal
            first_num = n
        return None

    
    def get_current_signal(self, **kwargs):
        """optimization required"""
        df = kwargs.get("dataframe", self.dataframe)
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("last_signal_value", -1)]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[kwargs.get("second_last_signal_value", -2)]
        new_df = df[:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
    
    def strategy(self, **kwargs):
        df = self.dataframe
        last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["rsi_signal"] == "UP_REVERSE") | (df["rsi_signal"] == "DOWN_REVERSE")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name]
        new_df = new_df.iloc[::-1]
        signal = None
        data = pd.DataFrame()
        
        if last_sma_crossover.rsi_signal == "UP_REVERSE" and not new_df.empty:
            signal = "BUY"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if max(row.bidopen, row.bidclose) < row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        elif last_sma_crossover.rsi_signal == "DOWN_REVERSE" and not new_df.empty:
            signal = "SELL"
            valid_row = 0
            for row in new_df[1:4].itertuples():
                if min(row.bidopen, row.bidclose) > row.small_sma:
                    valid_row += 1
            if valid_row >= 2:
                data = new_df[last_sma_crossover.name : last_sma_crossover.name]
        
        if signal and not data.empty:
            return self.create_signal(data, signal=signal)
        return None
        

    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
    
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        


class Level3(BaseStrategy):
        
    def get_current_signal(self, df, **kwargs):
        df1 = df.copy()
        df2 = df.copy()
        strategy = SMAStrategy3Level1(stoploss=self.stoploss, target=self.target)
        strategy.dataframe = df1
        strategy.is_valid_dataframe()
        current_signal = strategy.get_current_signal()
        previous_signal = strategy.get_previous_signal()
        if kwargs.get("need_previous_signal"):
            setattr(self, "_previous_signal", previous_signal)
        exit_strategy = SMAStrategy3Exit(stoplos=self.stoploss, target=self.target)
        exit_strategy.dataframe = df2
        exit_strategy.entry_signal = next(pd.DataFrame(previous_signal).itertuples())
        exit_strategy.is_valid_dataframe()
        last_exit_signal= exit_strategy.get_signal()
        # print(last_exit_signal, "exit signal for last entry siignal")
        # print(current_signal, "current signal")
        # print(previous_signal, "last signal")
        try:
            current_signal_df = next(pd.DataFrame(current_signal).itertuples())
        except:
            current_signal_df = pd.DataFrame()
            
        try:
            last_signal_df = next(pd.DataFrame(previous_signal).itertuples())
        except:
            last_signal_df = pd.DataFrame()
            
        if last_exit_signal and last_exit_signal[2] == "TG" and current_signal:
            entry_diff = (pd.Timestamp(current_signal_df.Index) - pd.Timestamp(last_exit_signal[1])) / pd.Timedelta(minutes=1)
            if current_signal_df.entry_signal != last_signal_df.entry_signal and entry_diff <= 15:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
                return current_signal
        if last_exit_signal == (0,0,0) and current_signal:
            last_entry_signal_price = last_signal_df.bidclose
            current_entry_signal_price = current_signal_df.bidclose
            diff = 0
            if last_signal_df.entry_signal == "BUY":
                diff = current_entry_signal_price - last_entry_signal_price
            else:
                diff = last_entry_signal_price - current_entry_signal_price
            
            if diff <= -0.0003:
                return None
            elif diff >= 0.0003:
                sig_time, entry = list(*current_signal["entry_signal"].items())
                current_signal["entry_signal"] = {sig_time:last_signal_df.entry_signal}
        return current_signal
    
    def strategy(self):
        df = self.dataframe.copy()
        current_signal = self.get_current_signal(df, need_previous_signal=True)
        # sig_time, entry = list(*self._previous_signal["entry_signal"].items())
        # previous_signal = self.get_current_signal(df[:sig_time])
        # print(previous_signal)
        # actual_previous_signal = 
        
        return current_signal
    
st = SMAStrategy3Level1(stoploss=0.0010, target=0.0005)
test_data = data
st.dataframe = test_data
ic(test_data.iloc[-1])
st.is_valid_dataframe()
d = st.get_signal()
d


ic| test_data.iloc[-1]: bidopen       1.07288
                        bidclose      1.07298
                        bidhigh       1.07302
                        bidlow        1.07288
                        askopen       1.07301
                        askclose      1.07312
                        askhigh       1.07314
                        asklow        1.07301
                        tickqty     114.00000
                        Name: 2022-06-01 08:36:00, dtype: float64


{'bidopen': {Timestamp('2022-06-01 08:07:00'): 1.07336},
 'bidclose': {Timestamp('2022-06-01 08:07:00'): 1.07269},
 'bidhigh': {Timestamp('2022-06-01 08:07:00'): 1.07336},
 'bidlow': {Timestamp('2022-06-01 08:07:00'): 1.0726},
 'askopen': {Timestamp('2022-06-01 08:07:00'): 1.07351},
 'askclose': {Timestamp('2022-06-01 08:07:00'): 1.07282},
 'askhigh': {Timestamp('2022-06-01 08:07:00'): 1.07351},
 'asklow': {Timestamp('2022-06-01 08:07:00'): 1.07272},
 'tickqty': {Timestamp('2022-06-01 08:07:00'): 495},
 'max_bid': {Timestamp('2022-06-01 08:07:00'): 1.07336},
 'min_bid': {Timestamp('2022-06-01 08:07:00'): 1.0726},
 'small_sma': {Timestamp('2022-06-01 08:07:00'): 1.0722785714285714},
 'large_sma': {Timestamp('2022-06-01 08:07:00'): 1.072036},
 'rsi': {Timestamp('2022-06-01 08:07:00'): 61.84588608562065},
 'rsi_signal': {Timestamp('2022-06-01 08:07:00'): 'DOWN_REVERSE'},
 'sma_signal': {Timestamp('2022-06-01 08:07:00'): ''},
 'entry_price': {Timestamp('2022-06-01 08:07:00'): 1.07269},
 'e

In [33]:
class SMAStrategy2Exit(BaseStrategy):
    name = "smacrossover_with_smahit_strategy"
    description = """7,10 SMA"""
    
    def signal_found(self, data):
        return data.to7754w_dict()
    
    def strategy(self):
        """optimization required"""
        df = self.dataframe
        last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-1]
        second_last_sma_crossover = df.loc[(df["sma_signal"] == "UP_CROSSOVER") | (df["sma_signal"] == "DOWN_CROSSOVER")].iloc[-2]
        new_df = df[second_last_sma_crossover.name:last_sma_crossover.name ]
        # new_df = new_df.iloc[: , :-1]
        signal = None
        data = pd.DataFrame()
        amt = 0.00003
        
        if second_last_sma_crossover.sma_signal == "DOWN_CROSSOVER" and not new_df.empty:
            signal = "BUY"
            last_row = None
            for row in new_df.itertuples():
                if row.rsi_signal == "UP_REVERSE" and last_sma_crossover.sma_signal == "UP_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        elif second_last_sma_crossover.sma_signal == "UP_CROSSOVER" and not new_df.empty:
            signal = "SELL"
            for row in new_df.itertuples():
                if row.rsi_signal == "DOWN_REVERSE" and last_sma_crossover.sma_signal == "DOWN_CROSSOVER":
                    data = new_df[last_sma_crossover.name : last_sma_crossover.name]
                    break
        # if signal and not data.empty:
        #     d_date = data.index[0]
        #     d = data.to_dict()
        #     d["entry_price"] = d["bidclose"]
        #     d["entry_signal"] = {d_date: signal}
        #     return d
        return df
        # df["entry_price"] = df.bidopen
        # return self.signal_found(df.loc[(df["entry_signal"] == "BUY")].tail(1))
    
    def apply_indicator(self):
        super(self.__class__, self).apply_indicator()
        self.dataframe["small_sma"]=SMAIndicator(self.dataframe.bidclose, window=7).sma_indicator()
        self.dataframe["large_sma"]=SMAIndicator(self.dataframe.bidclose, window=10).sma_indicator()
        self.dataframe["rsi"] = rsi(self.dataframe.bidclose, 10)
        self.dataframe = self.dataframe.replace({np.nan: None})
        self.dataframe["rsi_signal"] = list(self.rsi_signal_maker())
        self.dataframe["sma_signal"] = list(self.sma_signal_maker())
        
                
    def sma_signal_maker(self):
        """function required more optimized logic"""
        last_row = None
        for row in self.dataframe.itertuples():
            if last_row == None:
                last_row = row
            if not all([row.small_sma, row.large_sma, last_row.small_sma, last_row.large_sma]):
                yield ""
                last_row = row
                continue
            elif row.small_sma < row.large_sma and last_row.small_sma > last_row.large_sma:
                yield "DOWN_CROSSOVER"
            elif row.small_sma > row.large_sma and last_row.small_sma < last_row.large_sma:
                yield "UP_CROSSOVER"
            else:
                yield ""
            last_row = row

        
    def rsi_signal_maker(self):
        last_value = 0
        for value in self.dataframe.rsi:
            if value == None:
                yield ""
                continue
            if last_value <= 30 and value > 30:
                yield "UP_REVERSE"
            elif last_value >=70 and value < 70:
                yield "DOWN_REVERSE"
            else:
                yield ""
            last_value = value

        

# strategy = SMAStrategy2()
# strategy.dataframe = data
# strategy.is_valid_dataframe()
# dd = strategy.get_signal()
# ie = 1.13077
# UP_CROSSOVER
# dd.head(2).loc[(dd["min_bid"] <= dd["small_sma"])]
# dd.drop(["askopen","askclose", "askhigh", "tickqty", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         ], axis=1)
# dd

In [12]:
test_data = data.copy() #[100:]
import backtest
import strategy
importlib.reload(strategy)
importlib.reload(backtest)
from backtest import BackTestStrategy
importlib.reload(strategy)
importlib.reload(backtest)

# exit_strategy=SMAExitLevel1,
sma_backtest = BackTestStrategy(SMAStrategy3Level1, test_data, target=0.0008,stoploss=0.0005, order_quantity=30000)
sma_backtest.start_backtesting()

  0%|          | 0/14269 [00:00<?, ?it/s]

packet queue is empty, aborting
|ERROR|2022-05-30 22:00:52,262|packet queue is empty, aborting
|ERROR|2022-05-30 22:02:29,608|Connection error: HTTPSConnectionPool(host='api-demo.fxcm.com', port=443): Read timed out. (read timeout=5)


'backtesting has been completed, please generate report'

In [ ]:
import strategy
importlib.reload(strategy)
import backtest
importlib.reload(backtest)
# kd = sma_backtest.get_report()
kd = sma_backtest.clean_report(dp)
# sma_backtest.signal_data
# kd["max_bid"] = kd[["bidopen","bidclose","bidlow","bidhigh"]].max(axis=1)2022-01-03 10:53:00
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
        "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
print(len(kd[kd["status"] == 0].pl))
print("False Signals")

ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| 

68 Total Stoploss
208 Total Target
125 Total TGP
0 Total SLP
181 Total SLPR
68 Total Stoploss
333 Total Target
Total Amount Earned 2740.4999999999745
From Date: 2022-05-01 22:28:00
To Date: 2022-05-13 20:51:00
(586, 11)


kd[kd["status"] == "SLPR"].pl.sum(): -2374.799999999997
ic| kd[kd["status"] == "TGP"].pl.sum(): 1334.3999999999621
ic| kd[kd["status"] == "SL"].pl.sum(): -1252.5000000000055
ic| kd[kd["status"] == "TG"].pl.sum(): 5033.400000000014


False Signals
4
False Signals


In [20]:
# 1 Total Stoploss
# 22 Total Target
# 102 Total TGP
# 0 Total SLP
# 43 Total SLPR
# 1 Total Stoploss
# 124 Total Target
# Total Amount Earned 1339.7999999999379
# From Date: 2021-12-01 01:14:00
# To Date: 2021-12-03 18:42:00
# (204, 14)
# ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
# ic| kd[kd["status"] == "SLPR"].pl.sum(): -488.40000000002885
# ic| kd[kd["status"] == "TGP"].pl.sum(): 1152.8999999999724
# ic| kd[kd["status"] == "SL"].pl.sum(): -16.80000000000348
# ic| kd[kd["status"] == "TG"].pl.sum(): 692.0999999999977
# False Signals
# 36
# kd.to_csv("dec_test_half.csv")
kd.drop(["max_bid","min_bid","sma_signal"],axis=1) #.to_csv("jan_report.csv") 
# diff = []
# for k in kd.itertuples():
#     try:
#         diff.append(k.exit_time - k.Index)
#     except:
#         diff.append(0)
# kd["diff"]= diff
# kd.drop(["max_bid","min_bid"],axis=1).head(50)
# kd[kd["status"] == "SLPR"].head(60).drop(["max_bid","min_bid","small_sma_8","large_sma_16","sma_signal_1","sma_signal"],axis=1)


,bidopen,rsi,entry_price,entry_signal,exit_price,exit_time,status,pl
2022-05-01 22:19:00,1.05428,54.550935,1.05474,BUY,0.00000,0,0,0.0
2022-05-01 22:28:00,1.05366,36.720081,1.05399,BUY,1.05452,2022-05-01 22:56:00,TGP,15.9
2022-05-01 22:30:00,1.05365,36.112771,1.05381,BUY,1.05452,2022-05-01 22:56:00,TGP,21.3
2022-05-01 22:56:00,1.05455,69.550617,1.05452,SELL,1.05478,2022-05-01 23:09:00,SLPR,-7.8
2022-05-01 23:54:00,1.05380,35.110071,1.05392,BUY,1.05457,2022-05-02 00:06:00,TGP,19.5
...,...,...,...,...,...,...,...,...
2022-05-13 19:49:00,1.03965,62.473651,1.04001,BUY,1.04026,2022-05-13 19:53:00,TGP,7.5
2022-05-13 19:53:00,1.04040,65.543356,1.04026,SELL,1.04061,2022-05-13 20:45:00,SLPR,-10.5
2022-05-13 20:40:00,1.04045,67.323014,1.04037,SELL,1.04088,2022-05-13 20:58:00,SL,-15.3
2022-05-13 20:43:00,1.04050,60.322815,1.04039,SELL,0.00000,0,0,0.0


In [38]:
k = pd.Timestamp(test_signal.Index) - pd.Timedelta(250, "m")
# data[str(k):test_signal.Index]
k < pd.Timestamp(test_signal.Index)

True

In [16]:
kd = sma_backtest.clean_report(dp)[["bidopen","bidclose","entry_price","entry_signal","exit_price","exit_time","status","pl"]]
print(len(list(filter(lambda k : k == "SL", kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k == "TG", kd.status))) , "Total Target")
print(len(list(filter(lambda k : k == "TGP", kd.status))) , "Total TGP")
print(len(list(filter(lambda k : k == "SLP", kd.status))) , "Total SLP")
print(len(list(filter(lambda k : k == "SLPR", kd.status))) , "Total SLPR")
print(len(list(filter(lambda k : k in ["SL", "SLP"], kd.status))) , "Total Stoploss")
print(len(list(filter(lambda k : k in ["TG","TGP"], kd.status))) , "Total Target")
print(f"Total Amount Earned {kd.pl.sum()}")
print(f"From Date: {kd.iloc[1].name}")
print(f"To Date: {kd.iloc[-1].name}")
# kd = kd.drop(["askopen","askclose", "askhigh", "bidclose","bidhigh", "bidclose", "bidlow", "asklow",
#         "rsi_signal","large_sma","small_sma","tickqty","pip"], axis=1)
print(kd.shape)
ic(kd[kd["status"] == "SLP"].pl.sum())
ic(kd[kd["status"] == "SLPR"].pl.sum())
ic(kd[kd["status"] == "TGP"].pl.sum())
ic(kd[kd["status"] == "SL"].pl.sum())
ic(kd[kd["status"] == "TG"].pl.sum())
print("False Signals")
len(kd[kd["status"] == 0].pl)

ic| kd[kd["status"] == "SLP"].pl.sum(): 0.0
ic| kd[kd["status"] == "SLPR"]

11 Total Stoploss
208 Total Target
125 Total TGP
0 Total SLP
181 Total SLPR
11 Total Stoploss
333 Total Target
Total Amount Earned 3797.6999999999725
From Date: 2022-05-01 22:28:00
To Date: 2022-05-13 20:51:00
(586, 8)


.pl.sum(): -2374.799999999997
ic| kd[kd["status"] == "TGP"].pl.sum(): 1334.3999999999621
ic| kd[kd["status"] == "SL"].pl.sum(): -195.30000000000715
ic| kd[kd["status"] == "TG"].pl.sum(): 5033.400000000014


False Signals


61

In [17]:
# dp[["exit_time", "entry_signal", "pl","entry_price","exit_price","status"]].to_csv("may2022_filter_str.csv")

# Backtesting Packages

In [160]:
from tqdm.notebook import trange, tqdm
import time

# Backtesting Code

## Settings

In [161]:
entryStrategy = None
exitStrategy = None
stoploss = None
target = None
quantity = 10000
backtesting_dataframe = pd.DataFrame()

#### Start Backtesting Code

In [171]:
# entryStrategy = entryStrategy(stoploss=stoploss, target=target)

# for row_number in trange(1,len(backtesting_dataframe)):
    